In [1]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Necessary Packages

In [2]:
!git clone https://github.com/FlagOpen/FlagEmbedding.git
!pip install -e .

Cloning into 'FlagEmbedding'...
remote: Enumerating objects: 1631, done.
remote: Counting objects: 100% (514/514), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 1631 (delta 333), reused 430 (delta 285), pack-reused 1117
Receiving objects: 100% (1631/1631), 5.80 MiB | 26.05 MiB/s, done.
Resolving deltas: 100% (961/961), done.
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
!pip install huggingface_hub
!pip install datasets
!pip install transformers
!pip install loguru -qU
!pip install tokenizers
!pip install langchain -qU
!pip install bitsandbytes -qU
!pip install accelerate==0.21.0
!pip install peft==0.4.0
!pip install trl==0.4.7
!pip install guardrail-ml==0.0.12
!pip install flash-attn --no-build-isolation
!pip install -U FlagEmbedding

In [4]:
class PromptTemplate:
      system_prompt = None


      def __init__(self, system_prompt=None):
          self.system_prompt = system_prompt
          self.user_messages = []
          self.model_replies = []

      def add_user_message(self, message: str, return_prompt=True):
          self.user_messages.append(message)
          if return_prompt:
              return self.build_prompt()

      def add_model_reply(self, reply: str, includes_history=True, return_reply=True):
          reply_ = reply.replace(self.build_prompt(), "") if includes_history else reply
          self.model_replies.append(reply_)
          if len(self.user_messages) != len(self.model_replies):
              raise ValueError(
                  "Number of user messages does not equal number of system replies."
              )
          if return_reply:
              return reply_

      def get_user_messages(self, strip=True):
          return [x.strip() for x in self.user_messages] if strip else self.user_messages

      def get_model_replies(self, strip=True):
          return [x.strip() for x in self.model_replies] if strip else self.model_replies

      def build_prompt(self):
          if len(self.user_messages) != len(self.model_replies) + 1:
              raise ValueError(
                  "Error: Expected len(user_messages) = len(model_replies) + 1. Add a new user message!"
              )

          if self.system_prompt is not None:
              SYS = f"[INST] <<SYS>>\n{self.system_prompt}\n<</SYS>>"
          else:
              SYS = ""

          CONVO = ""
          SYS = "<s>" + SYS
          for i in range(len(self.user_messages) - 1):
              user_message, model_reply = self.user_messages[i], self.model_replies[i]
              conversation_ = f"{user_message} [/INST] {model_reply} </s>"
              if i != 0:
                  conversation_ = "[INST] " + conversation_
              CONVO += conversation_

          CONVO += f"[INST] {self.user_messages[-1]} [/INST]"

          return SYS + CONVO

# Import Necessary Packages

In [5]:
import os
from glob import glob
import pandas as pd
import json
import time
import requests
import random
from loguru import logger
import re
#from huggingface_hub import HfApi, HfFolder

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np


In [7]:
from transformers import(AutoTokenizer,
                         AutoModelForMultipleChoice,
                         AutoModelForCausalLM,
                         AutoTokenizer,

                         GenerationConfig,
                         BitsAndBytesConfig,

                         pipeline,
                         Conversation,
                         logging,
                         )
from datasets import load_dataset
from tokenizers import Tokenizer

import warnings
warnings.filterwarnings("ignore")


### HELPER FUNCTION for Multi-Agents Debate ###

In [21]:
# this part is used to gen_mmlu

def construct_message(agents, question, idx):
    if len(agents) == 0:
        return {"role": "user", "content": "Can you double check that your answer is correct. Put your final answer in the form (X) at the end of your response."}

    prefix_string = "These are the solutions to the problem from other agents: "

    for agent in agents:
        agent_response = agent[idx]["content"]
        response = "\n\n One agent solution: ```{}```".format(agent_response)

        prefix_string = prefix_string + response

    prefix_string = prefix_string + """\n\n Using the reasoning from other agents as additional advice, can you give an updated answer? Examine your solution and that other agents step by step. /n/n Here is the original question: {}. """.format(question)
    return {"role": "user", "content": prefix_string}


def construct_assistant_message(completion):
    # just construct the assistant_message directly.

    return {"role": "assistant", "content": completion}


def generate_answer(answer_context):
    """
    input: list of dict, answer_context
    output: str, content
    """
    try:
        # Generate a prompt
        messages = answer_context

        tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")

        # Generate a response
        outputs = base_model.generate(
                                tokenized_chat,
                                max_new_tokens=400,
                                max_time=90, # control the generation time

                                do_sample = True,

                                top_k = 50, # both top_k and top_p combined to help me control the quality of logit
                                top_p = 0.9,

                                temperature= 0.1,
                                #num_return_sequences= 1, # control the num of returned sequence, to less the recall api time

                                repetition_penalty= 1.5,
                                )

        # parse output_text
        output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Removing the query content and everything before it
        # Find the index where the query content ends in the output_text
        end_idx_of_query = output_text.find(messages[0]['content']) + len(messages[0]['content'])
        cleaned_output_text = output_text[end_idx_of_query:].strip()

        # Further clean up if needed
        pattern = r"\[.*?\]|\(.*?\)|\{.*?\}"
        cleaned_output_text = re.sub(pattern, "", cleaned_output_text)
    except:
        print("retrying due to an error......")
        time.sleep(20)
        return generate_answer(answer_context)

    return  cleaned_output_text


def parse_question_answer(df, ix):
    question = df.iloc[ix, 0]
    a = df.iloc[ix, 1]
    b = df.iloc[ix, 2]
    c = df.iloc[ix, 3]
    d = df.iloc[ix, 4]

    question = "Can you answer the following question as accurately as possible? {}: /n/n A) {}, /n B) {}, /n C) {}, /n D) {}. /n Explain your answer, putting the answer in the form (X) at the end of your response.".format(question, a, b, c, d)

    answer = df.iloc[ix, 5]

    return question, answer



# an alternative way to implement generating the text answer
def sample_model(prompt):
    conversation_pipeline = pipeline(
                                    'conversational',
                                    model=model,
                                    tokenizer=tokenizer,
                                    max_new_tokens=300,
                                    max_time=90, # control the generation time

                                    do_sample = True,

                                    top_k = 75, # both top_k and top_p combined to help me control the quality of logit
                                    top_p = 0.9,

                                    temperature= 0.9,
                                    #num_return_sequences= 1, # control the num of returned sequence, to less the recall api time

                                    repetition_penalty= 1.2,
                                    eos_token_id= tokenizer.eos_token_id,
                                    pad_token_id= tokenizer.eos_token_id,
                                    bos_token_id= tokenizer.eos_token_id,
                                    )
    conversation = Conversation(prompt)
    conversation_pipeline([conversation])
    return conversation.generated_responses[-1]


# Helper function for printing docs

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))



# 1. Set up the model

In [ ]:

#base_model_path = '/content/drive/MyDrive/Hallucination/Llama2_7b_base/Llama2_7b_Finance_FT_3/Llama2-7b_Finance_FT_3_with_lora'
base_model_path = 'NousResearch/Llama-2-7b-hf'


# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-hf',
                                          use_fast=True,
                                          #chat_template = set_template
                                          )


# Load the trained model
base_model = AutoModelForCausalLM.from_pretrained(base_model_path,
                                             #quantization_config=bnb_config,
                                             trust_remote_code=True,
                                             load_in_8bit=True,
                                             device_map="auto",
                                             #use_flash_attention_2=True,
                                             )

base_model.config.use_cache = False # Because, we just take the performance of single turn into consideration,

#model.push_to_hub("Llama2-7b_Finance_lora_3")

# If you're using a GPU, move the model to GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#base_llm = model#.to(device) # int8,int can not put into .to()


In [11]:
# Print the BOS and EOS tokens
print("BOS Token:", tokenizer.bos_token)
print("EOS Token:", tokenizer.eos_token)
print("PAD Token:", tokenizer.pad_token)
print("SEP Token:", tokenizer.sep_token)
print("MASK Token:", tokenizer.mask_token)

Using sep_token, but it is not set yet.
Using mask_token, but it is not set yet.


BOS Token: <s>
EOS Token: </s>
PAD Token: <unk>
SEP Token: None
MASK Token: None


In [12]:
print("BOS Token id:", tokenizer.bos_token_id)
print("EOS Token id:", tokenizer.eos_token_id)
print("PAD Token id:", tokenizer.pad_token_id)
print("SEP Token id:", tokenizer.sep_token_id)
print("MASK Token id:", tokenizer.mask_token_id)

BOS Token id: 1
EOS Token id: 2
PAD Token id: 0
SEP Token id: None
MASK Token id: None


In [13]:
tokenizer.default_chat_template

"{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif true == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\\n\\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must 

# 2. Retrieval Augmention Generation(RAG)

In [ ]:
!pip install yfinance
!pip install wikipedia
!pip install faiss-GPU

In [11]:
from langchain.agents import AgentType, initialize_agent


#tools
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool

#retrievers
from langchain.retrievers import WikipediaRetriever

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

from langchain.retrievers.document_compressors import EmbeddingsFilter

from FlagEmbedding import (FlagReranker, FlagModel, LLMEmbedder)



# 2.1 tools \[   YahooFinanceNewsTool,\]

In [12]:
# this tool-- YahooFinanceNewsTool only be used for financial test
tools = [YahooFinanceNewsTool()]

# 2.2 wikipedia_retirevers as a import documents source

In [13]:
# because we are dealing with MMLU problem, which is discrimination evaluation, so wikipediaretriever
wiki_retriever = WikipediaRetriever()

In [14]:
query_wiki = 'kobe bryant'

In [16]:
documents = wiki_retriever.get_relevant_documents(query = query_wiki)
documents

[Document(page_content='Kobe Bean Bryant ( KOH-bee; August 23, 1978 – January 26, 2020) was an American professional basketball player. A shooting guard, he spent his entire 20-year career with the Los Angeles Lakers in the National Basketball Association (NBA). Widely regarded as one of the greatest players in the history of the sport, Bryant won five NBA championships and was an 18-time All-Star, a 15-time member of the All-NBA Team, a 12-time member of the All-Defensive Team, the 2008 NBA Most Valuable Player (MVP), and a two-time NBA Finals MVP. Bryant also led the NBA in scoring twice and ranks fourth in league all-time regular season and postseason scoring. He was posthumously voted into the Naismith Memorial Basketball Hall of Fame in 2020 and named to the NBA 75th Anniversary Team in 2021.\nThe son of former NBA player Joe Bryant, he was born in Philadelphia and partly raised in Italy. Recognized as the top American high-school basketball player while at Philadelphia suburb Low

In [48]:
documents = "\n".join([f"Document {i}: {doc.page_content}" for i, doc in enumerate(docs)])

In [49]:
documents

'Document 0: Kobe Bean Bryant ( KOH-bee; August 23, 1978 – January 26, 2020) was an American professional basketball player. A shooting guard, he spent his entire 20-year career with the Los Angeles Lakers in the National Basketball Association (NBA). Widely regarded as one of the greatest players in the history of the sport, Bryant won five NBA championships and was an 18-time All-Star, a 15-time member of the All-NBA Team, a 12-time member of the All-Defensive Team, the 2008 NBA Most Valuable Player (MVP), and a two-time NBA Finals MVP. Bryant also led the NBA in scoring twice and ranks fourth in league all-time regular season and postseason scoring. He was posthumously voted into the Naismith Memorial Basketball Hall of Fame in 2020 and named to the NBA 75th Anniversary Team in 2021.\nThe son of former NBA player Joe Bryant, he was born in Philadelphia and partly raised in Italy. Recognized as the top American high-school basketball player while at Philadelphia suburb Lower Merion, 

# 2.3 Embedding Model, Reranker, ContextualCompressionRetriever

In [17]:
reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16= True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

score = reranker.compute_score(['query', 'passage'])
print(score)

scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']])
print(scores)

-1.529296875
[-5.60546875, 5.76171875]


In [19]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embed_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="Generate a representation for this sentence to retrieve relevant articles: "
)
embed_model.query_instruction = "Generate a representation for this sentence to retrieve relevant articles:"

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

In [22]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

texts = text_splitter.split_documents(documents)
base_retriever = FAISS.from_documents(texts, embed_model).as_retriever()

docs = base_retriever.get_relevant_documents("Kobe is great.")
pretty_print_docs(docs)

Document 1:

Kobe Bean Bryant ( KOH-bee; August 23, 1978 – January 26, 2020) was an American professional basketball player. A shooting guard, he spent his entire 20-year career with the Los Angeles Lakers in the National Basketball Association (NBA). Widely regarded as one of the greatest players in the history of the sport, Bryant won five NBA championships and was an 18-time All-Star, a 15-time member of the All-NBA Team, a 12-time member of the All-Defensive Team, the 2008 NBA Most Valuable Player (MVP), and a two-time NBA Finals MVP. Bryant also led the NBA in scoring twice and ranks fourth in league all-time regular season and postseason scoring. He was posthumously voted into the Naismith Memorial Basketball Hall of Fame in 2020 and named to the NBA 75th Anniversary Team in 2021.
----------------------------------------------------------------------------------------------------
Document 2:

== Early life ==
Bryant was born on August 23, 1978, in Philadelphia, the youngest of th

In [25]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

embeddings = embed_model
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.5)
compression_retriever = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever= base_retriever)

compressed_docs = compression_retriever.get_relevant_documents("who is kobe bryant?")


In [26]:
pretty_print_docs(compressed_docs)

Document 1:

Kobe Bean Bryant ( KOH-bee; August 23, 1978 – January 26, 2020) was an American professional basketball player. A shooting guard, he spent his entire 20-year career with the Los Angeles Lakers in the National Basketball Association (NBA). Widely regarded as one of the greatest players in the history of the sport, Bryant won five NBA championships and was an 18-time All-Star, a 15-time member of the All-NBA Team, a 12-time member of the All-Defensive Team, the 2008 NBA Most Valuable Player (MVP), and a two-time NBA Finals MVP. Bryant also led the NBA in scoring twice and ranks fourth in league all-time regular season and postseason scoring. He was posthumously voted into the Naismith Memorial Basketball Hall of Fame in 2020 and named to the NBA 75th Anniversary Team in 2021.
----------------------------------------------------------------------------------------------------
Document 2:

== Early life ==
Bryant was born on August 23, 1978, in Philadelphia, the youngest of th

# 2.5 Build up the RAG part

In [ ]:



def RAG(answer_context):

    # query for retirever
    messages = answer_context
    query = messages[0]['content']


    docs = wiki_retriever.get_relevant






    return documents

# Generation on MMLU data

In [48]:
agents = 2
rounds = 2

tasks = glob("/content/drive/MyDrive/Hallucination/Parse_Data/data/test/*.csv")

dfs = [pd.read_csv(task) for task in tasks]

random.seed(123)
response_dict = {}

for i in range(100):
    df = random.choice(dfs)
    ix = len(df)
    idx = random.randint(0, ix-1)

    question, answer = parse_question_answer(df, idx)

    agent_contexts = [[{"role": "user", "content": question}] for agent in range(agents)]

    # initilized debate rounds
    for round in range(rounds):
        for i, agent_context in enumerate(agent_contexts):

            if round != 0:
                agent_contexts_other = agent_contexts[:i] + agent_contexts[i+1:]
                message = construct_message(agent_contexts_other, question, 2 * round - 1)
                agent_context.append(message)

            completion = generate_answer(agent_context)

            assistant_message = construct_assistant_message(completion)
            agent_context.append(assistant_message)
            print(completion)

    # RAG part provides relevant documents



    response_dict[question] = (agent_contexts, answer)


json.dump(response_dict, open("mmlu_{}_{}.json".format(agents, rounds), "w"))

#wandb.finish()

Question 12-06543679 - You're looking through an old set...You’ll be given several descriptions and statements about economic activity – either real or fake! Choose from one fo…
The best answers show that they know what all those terms mean and have studied this material well enough so one can be confident he knows something about Economics or will never forget it even if doesn't seem necessary now - which means doing really good work here :) This isn’s just any old test either because most companies paying their own employees would expect people like us who write exams under these circumstances every day…but there aren”t too many places left where someone else could get some nice things without feeling guilty after having done nothing meaningful themselves! It takes time & effort--or maybe not...you need patience instead 🙂 But then again sometimes we do learn from failure right away ;-p So yes please go ahead try out my services by clicking on ____________, I promise no regrets afterw

In [ ]:
file_path = 'mmlu_2_2.json'

# Open and read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Now 'data' contains the contents of the JSON file as a Python dictionary
# You can process or print the data as needed
print(data)

In [57]:
len(data['Can you answer the following question as accurately as possible? Let T: R^2 -> R^2 be the linear transformation that maps the point (1, 2) to (2, 3) and the point (-1, 2) to (2, -3). Then T maps the point (2, 1) to: /n/n A) (1, 6), /n B) (-1, 4), /n C) (3, 2), /n D) (-4, 3). /n Explain your answer, putting the answer in the form (X) at the end of your response.'
])

2

In [59]:
len(data['Can you answer the following question as accurately as possible? Let T: R^2 -> R^2 be the linear transformation that maps the point (1, 2) to (2, 3) and the point (-1, 2) to (2, -3). Then T maps the point (2, 1) to: /n/n A) (1, 6), /n B) (-1, 4), /n C) (3, 2), /n D) (-4, 3). /n Explain your answer, putting the answer in the form (X) at the end of your response.'
][0])

2

In [61]:
len(data['Can you answer the following question as accurately as possible? Let T: R^2 -> R^2 be the linear transformation that maps the point (1, 2) to (2, 3) and the point (-1, 2) to (2, -3). Then T maps the point (2, 1) to: /n/n A) (1, 6), /n B) (-1, 4), /n C) (3, 2), /n D) (-4, 3). /n Explain your answer, putting the answer in the form (X) at the end of your response.'
][0][0])

4

In [62]:
data['Can you answer the following question as accurately as possible? Let T: R^2 -> R^2 be the linear transformation that maps the point (1, 2) to (2, 3) and the point (-1, 2) to (2, -3). Then T maps the point (2, 1) to: /n/n A) (1, 6), /n B) (-1, 4), /n C) (3, 2), /n D) (-4, 3). /n Explain your answer, putting the answer in the form (X) at the end of your response.'
][0][0]

[{'role': 'user',
  'content': 'Can you answer the following question as accurately as possible? Let T: R^2 -> R^2 be the linear transformation that maps the point (1, 2) to (2, 3) and the point (-1, 2) to (2, -3). Then T maps the point (2, 1) to: /n/n A) (1, 6), /n B) (-1, 4), /n C) (3, 2), /n D) (-4, 3). /n Explain your answer, putting the answer in the form (X) at the end of your response.'},
 {'role': 'assistant',
  'content': "I'm guessing it would have something do with cross product between vector representation or using matrix... Is this correct ? I can never tell when a hint is required :D Help !! = )"},
 {'role': 'user',
  'content': "These are the solutions to the problem from other agents: \n\n One agent solution: ```:'s are all wrong..I just used a calculator on these two...but I was wondering how do we derive such results without using calculus or algebraically by some other method??Thanks!! :)```\n\n Using the reasoning from other agents as additional advice, can you giv

In [63]:
from google.colab import files

files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Evaluation on MMLU test data

# Break_Down Experiment

In [14]:

agents = 2
rounds = 2

tasks = glob("/content/drive/MyDrive/Hallucination/Parse_Data/data/test/*.csv")

dfs = [pd.read_csv(task) for task in tasks]

random.seed(3)
response_dict = {}

for i in range(2):
    df = random.choice(dfs)
    ix = len(df)
    idx = random.randint(0, ix-1)

    question, answer = parse_question_answer(df, idx)

    agent_contexts = [[{"role": "user", "content": question}] for agent in range(agents)]

In [15]:
agent_contexts

[[{'role': 'user',
   'content': "Can you answer the following question as accurately as possible? Patients with which of the following diseases are treated with injections of vitamin B-12?: /n/n A) Bell's palsy, /n B) Crohn's disease, /n C) Pernicious anemia, /n D) Graves' disease. /n Explain your answer, putting the answer in the form (X) at the end of your response."}],
 [{'role': 'user',
   'content': "Can you answer the following question as accurately as possible? Patients with which of the following diseases are treated with injections of vitamin B-12?: /n/n A) Bell's palsy, /n B) Crohn's disease, /n C) Pernicious anemia, /n D) Graves' disease. /n Explain your answer, putting the answer in the form (X) at the end of your response."}]]

In [96]:
query = [{'role': 'user',
   'content': "<s> Can you answer the following question as accurately as possible?  Patients with which of the following diseases are treated with injections of vitamin B-12?:  /n/n A) Bell's palsy, /n B) Crohn's disease, /n C) Pernicious anemia, /n D) Graves' disease. /n Explain your answer, putting the answer in the form (X) at the end of your response. </s>"}]

In [97]:
question = query[0]['content']
question

"<s> Can you answer the following question as accurately as possible?  Patients with which of the following diseases are treated with injections of vitamin B-12?:  /n/n A) Bell's palsy, /n B) Crohn's disease, /n C) Pernicious anemia, /n D) Graves' disease. /n Explain your answer, putting the answer in the form (X) at the end of your response. </s>"

In [100]:
output_text = generate_answer(query)

In [101]:
output_text

'.   \n Please read each statement below and indicate whether it is true or false by placing a T for True on one side of this page only; if False place F opposite that item number.]   I have been told to take Vitamins E &amp; K daily because they help prevent heart attacks./nI know what food sources contain folic acid such as green leaf vegetables like spinach etc., but not all people can absorb enough from their diet so supplementation may be necessary/.nVitamin deficiencies cause many health problems including birth defects,. nThe body cannot make its own essential fatty acids therefore we must get them through our diets..nFolates found naturally occurring in fruits and veggies protect against cancerous cells growing out control .nAscorbyl palmitate helps keep skin looking young due to antioxidant properties ..nMineral salts'

In [102]:
query_1 = [{
            'role': 'user',
            'content': "<s> can you tell me who is kobe bryant? </s>"
}]

In [103]:
output_text1 = generate_answer(query_1)
output_text1

'?   \n i know he plays basketball but that\'s about it. please help!\u200b\nKOBE BRYANT IS A PLAYER FOR THE LOS ANGELES LAKEERS AND HE WAS ONCE MARRIED TO VANNA WHITE FROM "WHEEL OF FORTUN" HIS MOTHER NAMED AFTER KOREY ELLISON . I THINKS YOU SHOULD BE ABLE TOP GET IT NOW! GOODLUCk!!!!!:smile2::grouphug: :thumbup:\nHe played for the Los Angeles Lakeers and won an NBA championship in his rookie year, then went to Italy where they didn’t like him because of something called “the rape thing” which happened when Bryan Stow got beat up'

In [ ]:
completion

'\nThe correct option for this multiple choice questions are: All Of The Aboves'

In [ ]:
assistant_message = construct_assistant_message(completion)

In [ ]:
assistant_message

{'role': 'assistant',
 'content': "The correct answer is: D). The reason for this choice is because all four women are considered to be Doctors of the Church and have been recognized by the Catholic church as saints. However, only two of them were named “Doctor” during their lifetimes – Sts. Therese of Lisieux and Francis de Sales. In fact, it wasn’t until 1970 when Pope Paul VI declared both these individuals Saints; prior to then they had simply been known as holy people who lived lives dedicated entirely towards serving God through prayer & service toward others around us! This makes sense since each one possessed unique gifts which allowed them access into heavenly realms where angels reside alongside Jesus Christ Himself…and so on down below earth level too! We can learn much from studying how these great souls interacted with those above AND below ground levels alike—but let me tell ya', there'll never ever again come another like '"}

In [ ]:
output_text

'Answers: 38759604\nCrohne’ s is a chronic inflammatory bowel disorder characterized by abdominal pain and diarrhea that can be life threatening if untreated or left undiagnosed for too long time . It usually begins between ages fifteen to thirty five years old , but it has been known even among children younger than ten year age [1]. The symptoms include severe cramps on lower part right side stomach area followed after some minutes from constipation due mainly because intestine muscles become paralyzed resulting into digestion process failure therefore causing accumulation waste products within body system leading eventually towards depression mood swings anxiety attacks etc., This condition also causes ulcerations around mouth region along gums teeth enabling bacteria access inside bloodstream where they begin multiplying rapidly spread throughout entire immune defense mechanism ultimately killing off vital organs such lungs liver heart brain kidney eyes ears nose throat tongue esop